In [134]:
import numpy as np
import pandas as pd

In [136]:
df = pd.read_csv("Dataset_fertilizer.csv")

In [156]:
df.head(15)

,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,37,0,0,6
1,12,0,36,0
2,7,9,30,1
3,22,0,20,4
4,35,0,0,6
5,12,10,13,2
6,9,0,10,5
7,41,0,0,6
8,21,0,18,4
9,9,7,30,1


In [138]:
X = df.drop(columns=['Fertilizer Name'])
Y = df['Fertilizer Name']

In [139]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=0.7,shuffle=True,random_state=40)

In [144]:
from sklearn.preprocessing import StandardScaler
SS= StandardScaler()
X_train = SS.fit_transform(X_train)
X_test = SS.transform(X_test)

In [145]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=90,criterion = 'gini', random_state=40)
classifier.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=90, random_state=40)

In [146]:
Y_pred = classifier.predict(X_test)

In [148]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(Y_test,Y_pred)
print(cm)
accuracy_score(Y_test,Y_pred)

[[4 0 0 0 0 0 0]
 [0 8 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 5]]


1.0

In [149]:
from sklearn.preprocessing import LabelEncoder
encode_ferti = LabelEncoder()
df['Fertilizer Name']=encode_ferti.fit_transform(df['Fertilizer Name'])
#creating the dataframe
Fertilizer = pd.DataFrame(zip(encode_ferti.classes_,encode_ferti.transform(encode_ferti.classes_)),columns=['original','number'])
Fertilizer = Fertilizer.set_index('original')
Fertilizer

,number
original,
DAP,0
Fourteen-Thirty Five-Fourteen,1
Seventeen-Seventeen-Seventeen,2
Ten-Twenty Six-Twenty Six,3
Twenty Eight-Twenty Eight,4
Twenty-Twenty,5
Urea,6


In [150]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Fertilizer Name']),df['Fertilizer Name'],test_size=0.3,random_state=1)


In [151]:
rand = RandomForestClassifier(random_state = 40)
rand.fit(x_train,y_train)

RandomForestClassifier(random_state=40)

In [152]:
pred_rand = rand.predict(x_test)

In [153]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV


params = {
    'n_estimators':[300,450,600],
    'max_depth':[4,5,7],
    'min_samples_split':[2,5,8]
}
grid = GridSearchCV(rand,params,cv=3,verbose=3,n_jobs=-1)

grid.fit(x_train,y_train)

predict = grid.predict(x_test)

print('Params (best): ',grid.best_params_)
print('Score (best) : ',grid.best_score_)
print(classification_report(y_test,predict))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Params (best):  {'max_depth': 4, 'min_samples_split': 2, 'n_estimators': 300}
Score (best) :  0.9855072463768115
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         7

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [154]:
import pickle
pickle_out = open('model_classify.pkl', 'wb')
pickle.dump(grid,pickle_out)
pickle_out.close()

In [155]:
import pickle

classifier = pickle.load(open('model_classify.pkl', 'rb'))   # Loading the trained model
prediction = classifier.predict([[35, 5, 4]])           # Making prediction [N , P , K] values


prediction_messages = {
    0: "TEN-TWENTY SIX-TWENTY SIX",
    1: "Fourteen-Thirty Five-Fourteen",                  # Creating  a mapping of predictions to messages
    2: "Seventeen-Seventeen-Seventeen",
    3: "TWENTY-TWENTY",
    4: "TWENTY EIGHT-TWENTY EIGHT",
    5: "DAP"
}

message = prediction_messages.get(prediction[0], "UREA")
print(message)


UREA
